Author: Anna Liao, alliao@lbl.gov, August 2016

External dependencies.  Clone to local directory:

https://github.com/SoftwareDefinedBuildings/pybtrdb

https://github.com/SoftwareDefinedBuildings/uPMU-tools (only needed for get_peakc_dt())
In uPMU-tools, I modify btrdbwrapper.py and btrsearch.py to refer to a cloned repo as follows:
from pybtrdb import btrdb
In the code below, I refer to my modified module as btrdbutil2.
This is because 'pip install btrdb' wasn't working properly in my environment.

btrdb_capnp.py (only needed for get_peakc_dt())

# get_upmu_data(): Returns instantaneous P, Q, and voltage magnitude for specified datetime.

In [36]:
import sys
sys.path.append('/data/home/aliao/repos/')
from pybtrdb import btrdb
# https://github.com/SoftwareDefinedBuildings/pybtrdb
import numpy as np
from datetime import datetime
from pytz import timezone
pacific = timezone('US/Pacific')

def get_upmu_data(inputdt, upmu_path):
    """ Retrieves instantaneous P, Q, and voltage magnitude for specified datetime.
    
    Args:
        inputdt (datetime): timezone aware datetime object
        upmu_path (str): e.g., '/LBNL/grizzly_bus1/'
    Returns:
        {'P1': , 'Q1': , 'P2': , 'Q2': , 'P3': , 'Q3': , 
         'value_units': ('kW', 'kVAR'), 
         'L1MAG': , 'L2MAG': , 'L3MAG': }
    """
    
    bc = btrdb.HTTPConnection("miranda.cs.berkeley.edu")
    ur = btrdb.UUIDResolver("miranda.cs.berkeley.edu", "uuidresolver", "uuidpass", "upmu")
    
    # convert dt to nanoseconds since epoch
    epochns = btrdb.date(inputdt.strftime('%Y-%m-%dT%H:%M:%S'))
    
    # retrieve raw data from btrdb
    upmu_data = {}
    streams = ['L1MAG', 'L2MAG', 'L3MAG', 'C1MAG', 'C2MAG', 'C3MAG', 'L1ANG', 'L2ANG', 'L3ANG', 'C1ANG', 'C2ANG', 'C3ANG']
    for s in streams:
        pt = bc.get_stat(ur.resolve(upmu_path+s), epochns, epochns + int(9e6))
        upmu_data[s] = pt[0][2]
        
    output_dict = {}

    output_dict['P1'] = (upmu_data['L1MAG']*upmu_data['C1MAG']*np.cos(upmu_data['L1ANG'] - upmu_data['C1ANG']))*1e-3
    output_dict['Q1'] = (upmu_data['L1MAG']*upmu_data['C1MAG']*np.sin(upmu_data['L1ANG'] - upmu_data['C1ANG']))*1e-3

    output_dict['P2'] = (upmu_data['L2MAG']*upmu_data['C2MAG']*np.cos(upmu_data['L2ANG'] - upmu_data['C2ANG']))*1e-3
    output_dict['Q2'] = (upmu_data['L2MAG']*upmu_data['C2MAG']*np.sin(upmu_data['L2ANG'] - upmu_data['C2ANG']))*1e-3

    output_dict['P3'] = (upmu_data['L3MAG']*upmu_data['C3MAG']*np.cos(upmu_data['L3ANG'] - upmu_data['C3ANG']))*1e-3
    output_dict['Q3'] = (upmu_data['L3MAG']*upmu_data['C3MAG']*np.sin(upmu_data['L3ANG'] - upmu_data['C3ANG']))*1e-3

    output_dict['value_units'] = ('kW', 'kVAR')

    output_dict['L1MAG'] = upmu_data['L1MAG']
    output_dict['L2MAG'] = upmu_data['L2MAG']
    output_dict['L3MAG'] = upmu_data['L3MAG']
    return output_dict

# get_upmu_data(): example usage

In [33]:
import pprint
pp = pprint.PrettyPrinter()

inputdt = pacific.localize(datetime(2016, 8, 25, 19, 0, 0))
upmu_path = '/LBNL/grizzly_bus1/'

pp.pprint(get_upmu_data(inputdt, upmu_path))

{'L1MAG': 7287.4208984375,
 'L2MAG': 7299.921875,
 'L3MAG': 7318.2822265625,
 'P1': 7272.5364248477308,
 'P2': 2118.3817519608633,
 'P3': 6719.1867010705246,
 'Q1': -284.19075651498088,
 'Q2': -7184.1189935099919,
 'Q3': 3564.4269660296022,
 'value_units': ('kW', 'kVAR')}


# get_peakc_dt(): datetime of peak current in specified time range

In [37]:
import sys
sys.path.append('/data/home/aliao/repos/')
sys.path.append('/data/home/aliao/repos/btrdb-python')
sys.path.append('/data/home/aliao/repos/uPMU-tools')
# https://github.com/SoftwareDefinedBuildings/uPMU-tools

from btrdbutil2.btrdbwrapper import *
from btrdbutil2.btrsearch import *

import btrdb_capnp
from pybtrdb import btrdb
# https://github.com/SoftwareDefinedBuildings/btrdb
from uuid import UUID
import numpy as np
from datetime import datetime
from pytz import timezone
pacific = timezone('US/Pacific')

capnp_path = '/data/home/aliao/repos/btrdb/cpinterface/interface.capnp'
# capnp_path needs to be replaced with local path to file
bc = btrdb_capnp.BTrDBConnection("miranda.cs.berkeley.edu", 4410, capnp_path)
ur = btrdb.UUIDResolver("miranda.cs.berkeley.edu", "uuidresolver", "uuidpass", "upmu")

def btrdb_window(uu, start_ns, dur, seg_length):
    """ returns btrdb stat in specified time range with given resolution.
    
    Args:
        uu (str): btrdb uuid of upmu
        start_ns (int): in UTC, i.e., 1446328800000000000
        dur (int): in nanoseconds, i.e., int(15*60*1e9)
        seg_length (int): length of each data point, in nanoseconds
    Returns:
        
    """
    end_ns = start_ns + dur
    return bc.queryWindowValues(uu, start_ns, end_ns, seg_length)

def get_peakc_dt(startdt, enddt, upmuc_path):
    """ Returns datetime of peak current in range
    
    Args:
        startdt (datetime)
        enddt (datetime)
        upmu_path (str): specify path of current magnitude stream, e.g., '/LBNL/grizzly_bus1/C1MAG'
    """
    
    startstr = startdt.strftime('%Y-%m-%dT%H:%M:%S')
    endstr = enddt.strftime('%Y-%m-%dT%H:%M:%S')
    
    startns = btrdb.date(startstr)
    endns = btrdb.date(endstr)
    dur = endns - startns
    uu = UUID(ur.resolve(upmuc_path))
    
    C_day_stat = btrdb_window(uu, startns, dur, dur)
    C_day_max = C_day_stat[0][0]['max']
    
    btrdb_wrapper = BTrDBWrapper()
    searchTree = BTrSearch(btrdb_wrapper, upmuc_path)
    visitor = MaxComparator()
    searchTree.accept(visitor)
    # input to multi_resolution_search in UTC
    start_ts, pw = searchTree.multi_resolution_search(startstr, endstr, 'ylabel', 
                                                      threshold=0.999*C_day_max)
    # print localtime from start_ts
    # time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_ts[0]*1e-9))
    
    return datetime.fromtimestamp(start_ts[0]*1e-9)

# get_peakc_dt(): example usage

In [39]:
startdt = pacific.localize(datetime(2016, 8, 25, 19, 0, 0))
enddt = pacific.localize(datetime(2016, 8, 28, 10, 0, 0))
upmuc_path = '/LBNL/grizzly_bus1/C1MAG'

peakc_dt = get_peakc_dt(startdt, enddt, upmuc_path)
print peakc_dt
pp.pprint(get_upmu_data(peakc_dt, '/LBNL/grizzly_bus1/'))

2016-08-25 16:46:27.762479
{'L1MAG': 7301.4794921875,
 'L2MAG': 7309.59375,
 'L3MAG': 7329.84033203125,
 'P1': 6221.891885358129,
 'P2': 4880.8751030883795,
 'P3': 3802.0190323260827,
 'Q1': 3061.4652372511678,
 'Q2': -5206.8206898532153,
 'Q3': 6173.1913826077871,
 'value_units': ('kW', 'kVAR')}
